# Dynamic Tests

This jupyter notebook allows you to treat a video so that in its upper left corner appears the name of the label that the neural network thinks you are seeing at that moment.

In [1]:
# OS imports
import glob
import os,shutil
import re
# Keras imports
from keras.models import load_model
from keras.preprocessing import image as preprocessing
# Image processing imports
from PIL import Image, ImageFont, ImageDraw, ImageOps
import numpy as np
import cv2
# Other
import time

# My directories paths
commonPath = ""
videos = commonPath + "Dynamic Tests/Originals"
saveInDir = commonPath + "Dynamic Tests/Results"

Using TensorFlow backend.


## Video Map

In this cell we will map our video frames.

With a "for" i'm going to iterate and between X and Y frames i will assign a Z label.

In [3]:
mapVideoTest1=[]
numTotalFrames = 4282
for i in range(0, 385):
    mapVideoTest1.append(0)
for i in range(385, 640):
    mapVideoTest1.append(1)
for i in range(640, 1260):
    mapVideoTest1.append(2)
for i in range(1260, 2950):
    mapVideoTest1.append(4)
for i in range(2950, 3600):
    mapVideoTest1.append(5)
for i in range(3600, 3980):
    mapVideoTest1.append(6)
for i in range(3980, numTotalFrames):
    mapVideoTest1.append(3)
    
mapVideoTest2=[]
numTotalFrames = 5512
for i in range(0, 385):
    mapVideoTest2.append(0)
for i in range(385, 1100):
    mapVideoTest2.append(1)
for i in range(1100, 2050):
    mapVideoTest2.append(2)
for i in range(2050, 4200):
    mapVideoTest2.append(4)
for i in range(4200, 4900):
    mapVideoTest2.append(5)
for i in range(4900, 5350):
    mapVideoTest2.append(6)
for i in range(5350, numTotalFrames):
    mapVideoTest2.append(3)

## New video creation

With all this funtions you can select a video and use your neural network to predict frame by frame what the neural network think is watching.

In [4]:
def testVideo(videoPath, realLabelsMap):
    # We split the path and select nly the name of the .mp4
    videoName = videoPath.split('/')[2]
    cap = cv2.VideoCapture(videoPath)
    
    # We read the dimensions of the video and we minimize the new video size (less MB).
    frameWidth = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frameHeight = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    newVideoFramesSize = (int(frameWidth/5), int(frameHeight/5))
    
    # Temporal folder where we are going to sae the frames of the new video.
    try:
        os.stat('.temp')
    except:
        os.mkdir('.temp')
    
    totalTrues = 0
    frameNumber = 0
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret:
            #for every frame of the video...
            printAndRemove("Working on frame nº" + str(frameNumber))
            
            # Preprocessing of the image for the Neural Network.
            img = generateNeuralNetworkImage(frame, neuralNetworkImageSize)
            # Use Neural Network to predict the node.
            labelPredicted = translateNode(lookForBest(model.predict(img)))
            # We update the accuracy.
            if(translateNode(realLabelsMap[frameNumber]) == labelPredicted or realLabelsMap[frameNumber] == -1):
                totalTrues = totalTrues + 1
            # Write label in the new frame.
            newFrame = generateNewVideoFrame(
                frame, 
                newVideoFramesSize,
                labelPredicted, 
                translateNode(realLabelsMap[frameNumber]), 
                totalTrues/(frameNumber+1)
            )
            
            # Save new frame in temporal folder.
            cv2.imwrite(".temp/frame" + str(frameNumber) + ".jpg", cv2.cvtColor(newFrame, cv2.COLOR_BGR2RGB))
            frameNumber = frameNumber + 1
        else:
            break
    print("")
    
    # We create the new video with the new frames from the temporal folder (with the label).
    createNewVideoWithFrames(videoName, ".temp/*.jpg", newVideoFramesSize)
    # We move the video to the saveInDir folder.
    shutil.move(videoName, saveInDir)
    # Close all.
    cap.release()
    cv2.destroyAllWindows()
    # Remove temporal folder.
    shutil.rmtree('.temp/')
        
    # Print results.
    print("Video \"" + videoName + "\" ready")
    print("\t-True Positives: " + str(totalTrues))
    print("\t-Total Number Frames: " + str(frameNumber))
    print("\t-Accuracy: " + str(totalTrues/frameNumber))
    
# Print and remove the print
def printAndRemove(msg):
    print(msg, end="\r")
    time.sleep(0)
    
# Select the label most likely to be true
def lookForBest(prediction):
    best = 0
    categories = 6
    for i,probability in enumerate(prediction[0]):
        if probability > prediction[0][best]:
            best = i
    return best

# Use the neural network to predict what can be the image
def predictImg(img,realLabel,model):
    prediction = model.predict(img)
    return lookForBest(prediction)

# Translate the label number to his string
def translateNode(node):
    if node == 0:
        return "Hall - Resident Rooms"
    if node == 1:
        return "Hall - Stairs"
    if node == 2:
        return "Building - Dinning room"
    if node == 3:
        return "Building - Rooms"
    if node == 4:
        return "Parking"
    if node == 5:
        return "Building - Library"
    if node == 6:
        return "Building - Gym"
    return "Unknown"
    
# The image must be refined before used in the neural network. It must have the right dimensions.
def generateNeuralNetworkImage(image, size):
    img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    img = Image.fromarray(img).convert('RGB')
    img = img.rotate(180)
    img.thumbnail(neuralNetworkImageSize, Image.ANTIALIAS)
    img = np.rot90(img)
    img = Image.fromarray(img)
    img = np.expand_dims(img, axis=0)
    return img
    
# We create the new frame and we write the text we need on it.
def generateNewVideoFrame(frame, size, labelPredicted, labelReal, accuracy, ):
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = Image.fromarray(frame).convert('RGB')
    frame = frame.rotate(180)
    frame.thumbnail(size, Image.ANTIALIAS)
    frameDraw = ImageDraw.Draw(frame)
    frameDraw.text( (5, 5) , "Prediction:   " + labelPredicted,  font = ImageFont.truetype("arial.ttf", 12), fill=(255,255,255))
    frameDraw.text( (5, 20) , "Real Label: " + labelReal,  font = ImageFont.truetype("arial.ttf", 12), fill=(255,255,255))
    frameDraw.text( (5, 35) , "Accuracy: " + str(accuracy),  font = ImageFont.truetype("arial.ttf", 12), fill=(255,255,255))
    frame = np.array(frame)
    return frame

# We iterate in every frame from the temporal folder and write a new video.
def createNewVideoWithFrames(newVideoName, dirFrames, videoSize):
    video = cv2.VideoWriter(newVideoName, cv2.VideoWriter_fourcc(*'DIVX'), 30.0, videoSize)
    filenames = glob.glob(dirFrames)
    filenames.sort(key=recompile)

    for i,image in enumerate(filenames):
        printAndRemove("New Video - Working on frame nº" + str(i))
        video.write(cv2.imread(image))
    video.release()
    print("")

# Funtion to sort the images form the temporal folder.
def recompile(name):
    r= re.compile("(\d+).*").split(name)
    return [int(y) if y.isdigit() else y for y in r]

In [19]:
model = load_model(commonPath + 'Neural_Network.h5')
#neuralNetworkImageSize = (96, 54)
neuralNetworkImageSize = (128, 72)

# We execute the dinamic test code and create the new video.
testVideo(videos + "/test2.mp4", mapVideoTest2)

Working on frame nº5511
New Video - Working on frame nº5511
Video "test2.mp4" ready
	-True Positives: 2893
	-Total Number Frames: 5512
	-Accuracy: 0.5248548621190131
